# Scientific Computing Report

This report will outline the code produced for this module as a library able to compute limit cycles of ODE's and PDE's as well as confirming the steady-states of diffusive PDE's

# A brief summary of your software (30% of the marks)

Structure of the modules,
How inputs are taken,
Checks,
Outputs



# A description of the key software design decisions made (40% of the marks)

# A reflective learning log (30% of the marks)